In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import torch
import random
import numpy as np
from matplotlib import pyplot as plt

In [49]:
def load_data(path, maxlen=-1):
    s = []
    classes = set()
    for i in range(1, 2): # 300 - all datasets
        path_ = Path(path, f'{i}.txt')
        with path_.open('r') as f:
            df = pd.read_csv(f)
        classes = classes.union(set(df['event'].unique()))
        df['time'] = pd.to_datetime(df['time'])
        df['time'] = (df['time'] - df['time'][0]) / np.timedelta64(1,'D')
        if maxlen > 0:
            df = df.iloc[:maxlen]
        df = df.drop(['id', 'option1'], axis=1)
        s.append(df)

    classes = list(classes)
    class2idx = {clas: idx for idx, clas in enumerate(classes)}

    ss, Ts = [], []
    for i, df in enumerate(s):
        if s[i]['time'].to_numpy()[-1] < 0:
            continue
        s[i]['event'].replace(class2idx, inplace=True)
        
#         tens = torch.FloatTensor(np.vstack([s[i]['time'].to_numpy(), s[i]['event'].to_numpy()])).T
        
        # this variant worked for me 
        st = np.vstack([s[i]['time'].to_numpy(), s[i]['event'].to_numpy()])
        tens = torch.FloatTensor(st.astype(np.float32)).T
        
        if maxlen > 0:
            tens = tens[:maxlen]
        ss.append(tens)
        Ts.append(tens[-1, 0])

    Ts = torch.FloatTensor(Ts)

    return ss, Ts, class2idx

In [50]:
path = Path('../..', 'data', 'IPTV_Data')
ss, Ts, class2idx = load_data(path, maxlen=200)

### Test on shifted series

In [51]:
from soft_dtw import *

a = ss[0][:, 0]
b = ss[0][:, 0] + 10

criterion = Soft_DTW(gamma=1.0, norm=True)
loss = criterion(a, b)
print (loss)

tensor(9457.8701)


### Test on equal series

In [52]:
a = ss[0][:, 0]
b = ss[0][:, 0]

criterion = Soft_DTW(gamma=1.0, norm=True)
loss = criterion(a, b)
print (loss)

tensor(0.)


### Test on random data

In [54]:
a = torch.randn(5, 4)
b = torch.randn(6, 4)

criterion = Soft_DTW(gamma=1.0, norm=True)
loss = criterion(a, b)
print (loss)

tensor(27.5479)
